In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

device='cuda'

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=256)
test_dataloader = DataLoader(test_data, batch_size=256)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            # nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
# model = NeuralNetwork()
model = NeuralNetwork()
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [2]:
def train_loop(dataloder,model,loss_fn,optimizer):
    size=len(dataloder.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloder):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss =loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # if batch %100 ==0:
        #     loss,current =loss.item(),batch*batch_size+len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

test_loss, correct = 0, 0
def test_loop(dataloader,model,loss_fn):
    global test_loss, correct
    model.eval()
    size=len(dataloader.dataset)
    num_batches= len(dataloader)
    
    with torch.no_grad():
        for X, y in dataloader:
            X,y=X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = torch.load('model.pth')

In [5]:
import time
loss_fn = nn.CrossEntropyLoss()


epochs = 100
s=time.time()
k=20
last =-1
for t in range(epochs):
    
    learning_rate=k**1.4*1e-4
    if last>test_loss:
        k=k**0.9
    last=test_loss
    k+=3
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
e=time.time()

print(f"Done!:  {e-s}s")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.421220 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.430658 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.429831 

Epoch 4
-------------------------------


KeyboardInterrupt: 

In [215]:
torch.save(model,'model.pth')

In [223]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
import random
model.eval()

a=random.randint(0,10000)
print(a)
x, y = test_data[a][0], test_data[a][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    sum=pred[0].sum()
    for i,p in enumerate(pred[0]):
        print(f"{p:.1f}")
    # print(pred.size())
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

1053
5.1
4.1
11.1
6.6
10.6
-11.6
8.4
-19.4
3.6
-18.5
Predicted: "Pullover", Actual: "Coat"
